In [9]:
import os
from typing import List
from neo4j import GraphDatabase
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_neo4j import Neo4jChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# --- 1. CONFIGURATION ---
# Use the ABSOLUTE path to your vector database
CHROMA_PATH = r"C:\Users\shreyas\Aerothon\expirements\db_root\public\Knowledge_vectors"
MODEL_NAME = "llama3.1:8b-instruct-q4_K_M"

# Standard local Ollama port is 11434
OLLAMA_BASE_URL = "http://127.0.0.1:11434"

# Neo4j Credentials
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_AUTH = ("neo4j", "password")

# Initialize Local AI Components with explicit base_url to fix ResponseError
llm = ChatOllama(
    model=MODEL_NAME, 
    temperature=0, 
    num_ctx=8192, 
    base_url=OLLAMA_BASE_URL
)
embeddings = OllamaEmbeddings(
    model="nomic-embed-text", 
    base_url=OLLAMA_BASE_URL
)

# --- 2. SILENCE WARNINGS (Schema Initialization) ---
def initialize_neo4j_schema():
    """Defines relationship types in Neo4j to prevent 'UnknownRelationship' warnings."""
    driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)
    with driver.session() as session:
        session.run("""
            MERGE (s:Session {id: 'schema_init'})
            MERGE (m:Message {content: 'init', role: 'system'})
            MERGE (s)-[:LAST_MESSAGE]->(m)
            MERGE (m)-[:NEXT]->(m)
            DETACH DELETE s, m
        """)
    driver.close()

initialize_neo4j_schema()

# --- 3. CONNECT TO DATABASES ---
vectorstore = Chroma(
    persist_directory=CHROMA_PATH, 
    embedding_function=embeddings,
    collection_name="Knowledge_Store"  # Must match your ingestion script
)
# Search k=5 chunks (smaller chunks = more relevant segments)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# --- 4. PROMPTS & LOGIC ---
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

context_q_prompt = ChatPromptTemplate.from_messages([
    ("system", "Given the history, rephrase the user's last question into a standalone question."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer ONLY using the context below. If unsure, say you don't know.\n\nContext:\n{context}"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

def get_query(input_data):
    if input_data.get("chat_history") and len(input_data["chat_history"]) > 0:
        chain = context_q_prompt | llm | StrOutputParser()
        return chain.invoke(input_data)
    return input_data["input"]

# The core RAG Chain
rag_chain = (
    RunnablePassthrough.assign(
        context=RunnableLambda(get_query) | retriever | format_docs
    )
    | qa_prompt
    | llm
)

# --- 5. THE HYBRID CHAT FUNCTION ---
def chat(user_input: str, session_id: str = "user_session_1"):
    # Connect to persistent history in Neo4j
    history = Neo4jChatMessageHistory(
        url=NEO4J_URI,
        username=NEO4J_AUTH[0],
        password=NEO4J_AUTH[1],
        session_id=session_id
    )
    
    # 1. Generate standalone query based on history
    smart_query = get_query({"input": user_input, "chat_history": history.messages})
    
    # 2. Retrieve most relevant documents
    docs = retriever.invoke(smart_query)
    
    # 3. Generate response using the RAG chain
    response = rag_chain.invoke({"input": user_input, "chat_history": history.messages})
    
    # 4. Update memory in Neo4j
    history.add_user_message(user_input)
    history.add_ai_message(response.content)
    
    # 5. Extract metadata for citations
    sources = []
    for doc in docs:
        src = doc.metadata.get("source", "Unknown")
        pg = doc.metadata.get("page", "N/A")
        sources.append(f"- {src} (Page: {pg})")
    
    return f"{response.content}\n\n**Sources:**\n" + "\n".join(set(sources))

# --- TEST ---
if __name__ == "__main__":
    print(chat("can you summarize Chapter 2: Need Assessment, Formulation of Specifications and Procurement Planning .?"))

I don't know.

**Sources:**
- 1.DOE-Manual-for-Procurement-of-Goods.pdf (Page: 80)
- 1.DOE-Manual-for-Procurement-of-Goods.pdf (Page: 67)
- 1.DOE-Manual-for-Procurement-of-Goods.pdf (Page: 30)
- 1.DOE-Manual-for-Procurement-of-Goods.pdf (Page: 365)
- 1.DOE-Manual-for-Procurement-of-Goods.pdf (Page: 79)
